### Import Required Libraries and Set Up Environment Variables

In [104]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [105]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [106]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"



# Filter for movie reviews with "love" in the headline
query_headline = "love"
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (
    f"{url}"
    f"q={query_headline}"
    f"&fq={filter_query}"
    f"&sort={sort}"
    f"&fl={field_list}"
    f"&begin_date={begin_date}"
    f"&end_date={end_date}"
)

In [107]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
page = 0
iter_count = 20 # number of calls
for i in range(0, iter_count):

    # create query with a page number
    page_query = (f"{query_url}&page={i}&api-key={nyt_api_key}")
    # API results show 10 articles at a time
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(page_query).json()
    # print(json.dumps(reviews['response'], indent=4))
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)

    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
            # Print the page that was just retrieved
        print(f"Checked page {i}")

    except:
        # Print the page number that had no results then break from the loop
        print("No results on page: {i}")
        break





Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [108]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[0:4], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [109]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
nyt_reviews_pd = pd.json_normalize(reviews_list)
nyt_reviews_pd.head(2)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [110]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
nyt_reviews_pd['title'] = nyt_reviews_pd['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
nyt_reviews_pd.head(2)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo


In [111]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
nyt_reviews_pd['keywords'] = nyt_reviews_pd['keywords'].apply(extract_keywords)
nyt_reviews_pd.head(2)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo


In [112]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = nyt_reviews_pd['title'].to_list()
titles


['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [113]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [114]:
# Create an empty list to store the results
tmdb_movies_list = []

request_counter = 1
# Create a request counter to sleep the requests after a multiple
# of 50 requests
def sleep_check(counter):
    if counter % 50 == 0:
        print("I am sleeping")
        time.sleep(1)

# Loop through the titles
result = requests.get(url + titles[0] + tmdb_key_string).json()

for title in titles:
    # Check if we need to sleep before making a request
    sleep_check(request_counter)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    result = requests.get(url + title + tmdb_key_string).json()


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        moive_id = result['results'][0]['id']

        # Make a request for a the full movie details
        movie_query = f"https://api.themoviedb.org/3/movie/{moive_id}?api_key={tmdb_api_key}"

        # Execute "GET" request with url
        movie_data = requests.get(movie_query).json()
        # print(json.dumps(movie_data, indent=4))

        
        # Extract the genre names into a list
        genre_names = []
        for genre in movie_data['genres']:
            genre_names.append(genre['name'])

        # Extract the spoken_languages' English name into a list
        spoken_languages = []
        spoken_languages.append(movie_data['spoken_languages'][0]['english_name'])


        # Extract the production_countries' name into a list
        production_countries = []
        production_countries.append(movie_data['production_countries'][0]['name'])


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            'title': movie_data['title'],
            'original_title': movie_data['original_title'],
            'budget': movie_data['budget'],
            'original_language': movie_data['original_language'],
            'homepage': movie_data['homepage'],
            'overview': movie_data['overview'],
            'popularity': movie_data['popularity'],
            'runtime': movie_data['runtime'],
            'revenue': movie_data['revenue'],
            'release_date': movie_data['release_date'],
            'vote_average': movie_data['vote_average'],
            'vote_count': movie_data['vote_count'],
            'genres':genre_names,
            'spoken_languages':spoken_languages,
            'production_countries': production_countries
        })
        # Print out the title that was found
        print(f"found {title}")
    except:
        print(f"There was an error retrieving movie data with {title}")



found The Attachment Diaries
There was an error retrieving movie data with What’s Love Got to Do With It?’ Probably a Lo
found You Can Live Forever
found A Tourist’s Guide to Love
found Other People’s Children
found One True Loves
found The Lost Weekend: A Love Story
found A Thousand and One
found Your Place or Mine
found Love in the Time of Fentanyl
found Pamela, a Love Story
found In From the Side
found After Love
found Alcarràs
found Nelly & Nadine
found Lady Chatterley’s Lover
found The Sound of Christmas
found The Inspection
found Bones and All
found My Policeman
found About Fate
found Waiting for Bojangles
found I Love My Dad
found A Love Song
found Alone Together
found Art of Love
found The Wheel
found Thor: Love and Thunder
found Both Sides of the Blade
found Fire of Love
found Love & Gelato
found Stay Prayed Up
found Benediction
found Dinner in America
There was an error retrieving movie data with In a New York Minute
found Anaïs in Love
found I Love America
found See You Then

In [115]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[0:4], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.802,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
 

In [116]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df.head(2)

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.802,102,0,2021-10-07,3.0,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",33.662,96,15055,2023-03-24,6.6,37,"[Drama, Romance]",[English],[Canada]


### Merge and Clean the Data for Export

In [117]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(tmdb_df,nyt_reviews_pd, on='title')
merged_df.head(2)

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.802,102,0,2021-10-07,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",33.662,96,15055,2023-03-24,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None


In [118]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix=['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ['[',']','\'']

# Loop through the list of columns to fix
for colum in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[colum] = merged_df[colum].astype('string')

    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[colum] = merged_df[colum].str.replace(char, '')


# Display the fixed DataFrame
merged_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.802,102,0,2021-10-07,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",33.662,96,15055,2023-03-24,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,14.032,100,37820,2023-04-07,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,6.002,95,77145,2023-04-13,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,15.331,116,3395595,2023-03-31,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,The Last Face,The Last Face,0,en,,"Miguel, a heroic Spanish doctor, puts himself ...",19.410,130,0,2017-01-11,...,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
121,Lost in Paris,Paris pieds nus,0,fr,,Fiona visits Paris for the first time to assis...,8.968,83,0,2017-01-14,...,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
122,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,5.978,103,0,2016-12-02,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
123,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",27.723,111,413844,2017-03-10,...,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None


In [119]:
# Drop "byline.person" column
if 'byline.person' in merged_df.columns:
    merged_df = merged_df.drop(columns=['byline.person'])
merged_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.802,102,0,2021-10-07,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",33.662,96,15055,2023-03-24,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,14.032,100,37820,2023-04-07,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,6.002,95,77145,2023-04-13,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,15.331,116,3395595,2023-03-31,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,The Last Face,The Last Face,0,en,,"Miguel, a heroic Spanish doctor, puts himself ...",19.410,130,0,2017-01-11,...,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,None
121,Lost in Paris,Paris pieds nus,0,fr,,Fiona visits Paris for the first time to assis...,8.968,83,0,2017-01-14,...,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,None
122,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,5.978,103,0,2016-12-02,...,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,None
123,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",27.723,111,413844,2017-03-10,...,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,None


In [120]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(drop=True, inplace=True)
merged_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.802,102,0,2021-10-07,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",33.662,96,15055,2023-03-24,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,14.032,100,37820,2023-04-07,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,6.002,95,77145,2023-04-13,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,15.331,116,3395595,2023-03-31,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,The Mountain Between Us,The Mountain Between Us,35000000,en,http://MountainBetweenUs.com,Stranded on a mountain after a tragic plane cr...,66.713,109,62832209,2017-10-05,...,272,Review: Love in a Cold Climate in ‘The Mountai...,None,None,The Mountain Between Us,None,None,None,By Jeannette Catsoulis,None
118,The Last Face,The Last Face,0,en,,"Miguel, a heroic Spanish doctor, puts himself ...",19.410,130,0,2017-01-11,...,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,None
119,Lost in Paris,Paris pieds nus,0,fr,,Fiona visits Paris for the first time to assis...,8.968,83,0,2017-01-14,...,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,None
120,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,5.978,103,0,2016-12-02,...,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,None


In [121]:
# Export data to CSV without the index

folder = 'output/'
name = 'my_collected_data.csv'
file = os.path.join(folder, name)
merged_df.to_csv(file, index=False)
